In [2]:
from datasets import load_dataset
import torch
from transformer_lens import HookedTransformer
from transformers import AutoModelForCausalLM
model_name = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# model = HookedTransformer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
raise Exception("Stop here")
camel_ai_bio_hf = load_dataset("camel-ai/biology", split="train")
message1s = [x["message_1"] for x in camel_ai_bio_hf]
message2s = [x["message_2"] for x in camel_ai_bio_hf]
assert len(message1s) == len(message2s)
chats = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message1s[i]},
        {"role": "assistant", "content": message2s[i]}
    ]
    for i in range(len(message1s))
]
print(chats[0]) # Debug

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
from transformers import AutoTokenizer
from llama_utils import get_llama_tokens
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
llama_tokens = get_llama_tokens(tokenizer)
tokenizer.pad_token_id = llama_tokens.eos_id
tokenizer.padding_side = "right"
tokenized_chats = tokenizer.apply_chat_template(chats, tokenize=True, return_tensors="pt", padding=True)
# print(tokenized_chats[0]) # Debug
# tokenized_chats = tokenizer.batch_encode_plus(tokenized_chats, add_special_tokens=False)
# print(tokenized_chats[0]) # Debug

In [8]:
# tokens = tokenized_chats.input_ids
# print(tokens)
# print(tokens[0])
# print(tokens.shape)

In [9]:
import torch
tokens = tokenized_chats
print(type(tokens))
print(type(tokens[0]))
print(type(tokens[0][0]))
print("="*100)
print(len(tokens))
print(max(len(t) for t in tokens), min(len(t) for t in tokens))
tokens = torch.tensor(tokens, dtype=torch.long)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
20000
1232 1232


/tmp/ipykernel_1718443/630375346.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens, dtype=torch.long)
